In [381]:
import pandas as pd

In [382]:
base_df = pd.read_parquet('../bin/scraped_latin_texts_words.parquet')

In [384]:
word_df = base_df.copy()
word_df = pd.DataFrame(word_df.groupby('word')['count'].sum()).reset_index()
word_df['overall_frequency'] = word_df['count'] / sum(word_df['count'])
word_df['overall_frequency_rank'] = word_df['count'].rank(method='max', ascending=False).astype(int)
word_df = word_df.sort_values(by='overall_frequency_rank', ascending=True).reset_index().drop(columns='index')

merge_df = base_df.copy()
merge_df = pd.DataFrame(merge_df['word'].value_counts())
merge_df['cross_text_count'] = merge_df['word']
merge_df['word'] = merge_df.index
merge_df = merge_df.reset_index().drop(columns='index')

word_df = word_df.merge(merge_df, on='word', how='outer')

word_df['cross_text_frequency'] = word_df['cross_text_count'] / len(base_df.groupby(['author', 'title'])['title'])
word_df['cross_text_frequency_rank'] = word_df['cross_text_count'].rank(method='max', ascending=False).astype(int)

merge_df = base_df.copy()
merge_df = pd.DataFrame(merge_df.groupby(['author', 'word'])['count'].sum()).reset_index()
merge_df = pd.DataFrame(merge_df['word'].value_counts())
merge_df['cross_author_count'] = merge_df['word']
merge_df['word'] = merge_df.index
merge_df = merge_df.reset_index().drop(columns='index')

word_df = word_df.merge(merge_df, on='word', how='outer')

word_df['cross_author_frequency'] = word_df['cross_author_count'] / len(base_df['author'].unique())
word_df['cross_author_frequency_rank'] = word_df['cross_author_count'].rank(method='max', ascending=False).astype(int)

word_df['zipf_value'] = word_df['overall_frequency_rank'] * word_df['overall_frequency']

In [ ]:
# export word_df as parquet to bin folder
# word_df.to_parquet('../bin/scraped_latin_texts_word_data.parquet', index=False, compression="gzip", engine="pyarrow")

In [387]:
text_df = base_df.copy()
text_df = text_df.groupby(['author', 'source', 'title']).agg({'count': 'sum', 'word': 'nunique'}).rename(columns={'count': 'text_overall_word_count', 'word': 'text_unique_word_count'}).reset_index()
text_df['text_overall_word_count_rank'] = text_df['text_overall_word_count'].rank(method='max', ascending=False).astype(int)
text_df['text_unique_word_frequency'] = text_df['text_unique_word_count'] / text_df['text_overall_word_count']
text_df['text_unique_word_frequency_rank'] = text_df['text_unique_word_frequency'].rank(method='max', ascending=False).astype(int)
text_df = text_df.iloc[:, [0, 1, 2, 3, 5, 4, 6, 7]]

In [ ]:
# export text_df as parquet to bin folder
# text_df.to_parquet('../bin/scraped_latin_texts_text_data.parquet', index=False, compression="gzip", engine="pyarrow")

In [390]:
author_df = base_df.copy()
author_df = author_df.groupby(['author']).agg({'count': 'sum', 'word': 'nunique'}).rename(columns={'count': 'author_overall_word_count', 'word': 'author_unique_word_count'}).reset_index()
author_df['author_overall_word_count_rank'] = author_df['author_overall_word_count'].rank(method='max', ascending=False).astype(int)
author_df['author_unique_word_frequency'] = author_df['author_unique_word_count'] / author_df['author_overall_word_count']
author_df['author_unique_word_frequency_rank'] = author_df['author_unique_word_frequency'].rank(method='max', ascending=False).astype(int)
author_df = author_df.iloc[:, [0, 1, 3, 2, 4, 5]]

In [ ]:
# export author_df as parquet to bin folder
# author_df.to_parquet('../bin/scraped_latin_texts_author_data.parquet', index=False, compression="gzip", engine="pyarrow")